# Make dataset to be general form

by transforming json into text <br>
by dividign data into training and test according to sentence <br>
training data를 만들 때, 공백도 하나의 token으로 인정할지 안할지도 생각해볼 것.

In [2]:
# after merging multiple json files
path_read_json_file = 'data/labeled-requirements.json'
path_write_text_file = "labeled-requirements.txt"

###################################################################################################
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import *
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import json
from pprint import pprint
import string
import re
import ast
import pickle
import random

result_data = []

# DEFINE
SENTENCE = "0"
COMPONENT = "1___component"
R_COMPONENT = "2___refinement_of_component"
ACTION = "3___action"
R_ACTION = "4___argument_of_action"
CONDITION = "5___condition"
PRIORITY = "6___priority"
MOTIVATION = "7___motivation"
ROLE = "8___role"
OBJECT = "9___object"
R_OBJECT = "a___refinement_of_object"
S_ACTION = "b___sub_action"
S_R_ACTION = "c___sub_argument_of_action"
S_PRIORITY = "d___sub_priority"
S_ROLE = "e___sub_role"
S_OBJECT = "f___sub_object"
S_R_OBJECT = "g___sub_refinement_of_object"

### LOAD DATA (Read Json)
with open(path_read_json_file) as json_data:
    data = json.load(json_data)

## Main Function (for transforming from json to text)

여기서 어느 하나 example 중에서 annotation, " " -> [ ] 로 바꿀 필요 있음. (에러)

* puntuation 중에서 . (점)을 삭제안하고 사용할 것이다. 즉, 문장 마지막 token에서 . (점)이 없으면 모두 추가하자.

In [3]:
def ann_tokenize(point, LABEL): # For distinguish whether string or list
    temp2D = []
    
    #print(point[SENTENCE])
    #print('ann_tokinaw!!!!!!!!!!!!', list(str(point[LABEL]))[0], type(str(point[LABEL])))

    # 배열과 string을 구분지어줄라고...
    # 인위적으로 첫글자가 '['이면 배열이라 생각하고 ast.literal_eval함수를 사용해 배열로 복귀시킨다.
    if list(str(point[LABEL]))[0] != '[':
        #print('---string', word_tokenize(str(point[LABEL])))
        return word_tokenize(str(point[LABEL]))
    else:
        arr = ast.literal_eval(str(point[LABEL]))
        #print('ast.literal:::::', arr)
        for chunk in arr:
            temp1D = word_tokenize(str(chunk))
            temp2D.append(temp1D)
        #print('---array', temp2D)
        return temp2D         

def ann_sentence(tok_sentence, phrase, p, ann_n_t_sen, check_t_sen):
    # tok_sentence: ['I','am','a','boy']
    # phrase: ['a','boy']
    # ann_n_t_sen: [16, 16, 16, 16]
    # check_t_sen: [0, 0, 0, 0]
    
    len_phrase = len(phrase)
    
#     print('phrase:', phrase)
    
    for i, token in enumerate(tok_sentence):
#         print('token', token)
        if tok_sentence[i] == phrase[0] and check_t_sen[i] == 0:
 
            if tok_sentence[i+len_phrase-1] == phrase[-1]:

                if check_t_sen[i+len_phrase-1] == 0:
                
                    for n in range(i, i+len_phrase):

                        ann_n_t_sen[n] = p
                        check_t_sen[n] = 1
                    
                    break
    
    return ann_n_t_sen, check_t_sen

In [4]:
def remove_preprocessing(point):
    
    # Remove 'be able to'
    point['0'] = re.sub(' be able to', '', point['0'])
    if type(point['6___priority']) != type([]):
        point['6___priority'] = re.sub(' be able to', '', point['6___priority'])
        point['6___priority'] = re.sub(' be able', '', point['6___priority'])
      
    # Remove 'it would be great/nice that'
    point['0'] = re.sub('it Would be great that ', '', point['0'])    
    point['0'] = re.sub('It would be great that ', '', point['0'])
    point['0'] = re.sub('It would be nice in the future that ', '', point['0'])
    point['0'] = re.sub('It would be nice that ', '', point['0'])
    
    # Remove 'be nice to'
    point['0'] = re.sub(' be nice to', '', point['0'])
    if type(point['6___priority']) != type([]):
        point['6___priority'] = re.sub(' be nice to', '', point['6___priority'])
        point['6___priority'] = re.sub(' be nice', '', point['6___priority'])    
    
    # Remove 'be great to'
    point['0'] = re.sub(' be great to', '', point['0'])
    if type(point['6___priority']) != type([]):
        point['6___priority'] = re.sub(' be great to', '', point['6___priority'])
        point['6___priority'] = re.sub(' be great', '', point['6___priority'])     
    
    # Remove 'like to'
    point['0'] = re.sub(' like to', '', point['0'])
    if type(point['6___priority']) != type([]):
        point['6___priority'] = re.sub(' like to', '', point['6___priority'])
        point['6___priority'] = re.sub(' like', '', point['6___priority'])     
    
    return point 

In [5]:
def substitute_preprocessing(point):
    
    # Remove 'be able to'
    point['0'] = re.sub('search for', 'search', point['0'])
    point['9___object'] = re.sub('for ', '', point['9___object'])

    return point 

In [6]:
point_cnt = 0
test_cnt = 0
ann_info = {0: '/component/', 1: '/refinement_of_component/', 2: '/action/', 3: '/refinement_of_action/',
            4: '/condition/', 5: '/priority/', 6: '/motivation/', 7: '/role/', 8: '/object/', 9: '/refinement_of_object/',
            10: '/sub_action/', 11: '/sub_argument_of_action/', 12: '/sub_priority/', 13: '/sub_role/', 14: '/sub_object/',
            15: '/sub_refinement_of_object/', 16: '/none/'}

with open(path_write_text_file, "w") as text_file:

    for point in data: # point: 1 example

#         print(point['0'])
#         print(point['6___priority'])
        #point = preprocessing(point)
        #point = substitute_preprocessing(point)
#         print(point['0'])
#         print(point['6___priority'])
#         print('\n')
        
        # 1. Tokenize All Annotations      
        tok_sentence = word_tokenize(str(point[SENTENCE]))
        tok_component = ann_tokenize(point, COMPONENT)
        tok_r_component = ann_tokenize(point, R_COMPONENT)
        tok_action = ann_tokenize(point, ACTION)
        tok_r_action = ann_tokenize(point, R_ACTION)
        tok_condition = ann_tokenize(point, CONDITION)
        tok_priority = ann_tokenize(point, PRIORITY)
        tok_motivation = ann_tokenize(point, MOTIVATION)
        tok_role = ann_tokenize(point, ROLE)
        tok_object = ann_tokenize(point, OBJECT)
        tok_r_object = ann_tokenize(point, R_OBJECT)
        tok_s_action = ann_tokenize(point, S_ACTION)
        tok_s_r_action = ann_tokenize(point, S_R_ACTION)
        tok_s_priority = ann_tokenize(point, S_PRIORITY)
        tok_s_role = ann_tokenize(point, S_ROLE)
        tok_s_object = ann_tokenize(point, S_OBJECT)
        tok_s_r_object = ann_tokenize(point, S_R_OBJECT)

        
  
        #print(tok_priority) 
        #print('\n')
        
        
        
        
        
        tok_collection = [tok_component] + [tok_r_component] + [tok_action] + [tok_r_action] + [tok_condition] \
                        + [tok_priority] + [tok_motivation] + [tok_role] + [tok_object] + [tok_r_object] + [tok_s_action] \
                        + [tok_s_r_action] + [tok_s_priority] + [tok_s_role] + [tok_s_object] + [tok_s_r_object]
        
#         print(tok_collection)
        #[['system'], ['have', 'a', 'MDI', 'form'], ['viewing'], [], [], ['shall'], [], [], [['graph'], ['data', 'table']], [], [], [], [], [], [], []]
        
        # initialize 16(none) for integer check array for token sentence 
        ann_num_t_sen = [16] * len(tok_sentence)
        check_t_sen = [0] * len(tok_sentence)
        
        # initialize 0 for binary check array for token annotations
        check_t_coll = []
        len_t_coll = []
        for i, ann_arr in enumerate(tok_collection):

            if len(ann_arr) == 0: # [] ann 정보가 없을 때
                check_t_coll.append([-1])
                len_t_coll.append(0)
            else: # ann 정보가 있을 때...
                if type(ann_arr[0]) == type('str'): # "shall be able"
                    if len(ann_arr) == 1:
                        check_t_coll.append([0])
                        len_t_coll.append(1)
                    else:
                        check_t_coll.append([0] * len(ann_arr))
                        len_t_coll.append(len(ann_arr))
                else: # 배열일 때 e.g. ["program name", "listing of required classes"]
                    temp_arr = []
                    
                    for j, ann in enumerate(ann_arr):
                        if len(ann) == 1:
                            temp_arr.append([0])
                            len_t_coll.append(1)
                        else:
                            temp_arr.append([0] * len(ann_arr[j]))
                            len_t_coll.append(len(ann_arr[j]))
#                             temp_arr.append([0] * len(ann_arr[j]))
#                             len_t_coll.append(len(ann_arr[j]))
                    check_t_coll.append(temp_arr)
                    


        # 재료들...
        sorted_len_arr = sorted(len_t_coll, reverse=True)

        #print(check_t_coll)
        #[[0], [0, 0, 0, 0], [0], [-1], [-1], [0], [-1], [-1], [[0], [0, 0]], [-1], [-1], [-1], [-1], [-1], [-1], [-1]]
        #print(len_t_coll)
        #[1, 4, 1, 0, 0, 1, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0]
        #print(sorted_len_arr)
        #[4, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        
#         print(ann_num_t_sen)
#         print(check_t_coll)
#         print(sorted_len_arr)
#         print("=====================================================================")        
        
        
        ### Annotation Copy 시작...
        # 첫 번째 목표 check_t_coll에 있는 모든 element들이 -1값을 가지도록 할 것.
        # 두 번째 목표 sorted(len_t_coll, reverse=True)를 참고해서 큰 len를 가진 phrase들부터 copy할 것.
        # 세 번째 목표 ann_num_t_sen에 모든 ann 번호 할당 할 것.

        for length in sorted_len_arr:
            
#             print(length)
#             print(check_t_coll)
            if length == 0:
                break
            else:
                
                for i, ann_arr in enumerate(tok_collection):
                    
                    if len(ann_arr) != 0:
                    
                        if type(ann_arr[0]) == type('str'):
#                             print('check_t_coll[0][0]', check_t_coll[i][0])
                            if check_t_coll[i][0] == 0: # 0: 아직 copy가 안된 ann
                                if len(ann_arr) == length:
#                                     print('here!!!')
                                    phrase = ann_arr
                                    ann_num_t_sen, check_t_sen = ann_sentence(tok_sentence, phrase, i, ann_num_t_sen, check_t_sen)
                                    check_t_coll[i] = [-1] * len(ann_arr)
                                    break


                        else:
                            for j, ann in enumerate(ann_arr):
                                
                                if check_t_coll[i][j][0] == 0: # 0: 아직 copy가 안된 ann
                                    if len(ann) == length:
#                                         print('here!!!')
                                        phrase = ann
                                        #list
                                        ann_num_t_sen, check_t_sen = ann_sentence(tok_sentence, phrase, i, ann_num_t_sen, check_t_sen)
                                        check_t_coll[i][j] = [-1] * len(ann)
                                        break
                
#         print(ann_num_t_sen)
#         print(check_t_coll)
#         print(sorted_len_arr)        
        
        #print(tok_sentence)
        
    
        #print(ann_num_t_sen)
        ###############################
        """ Text file에 write하기... """ 
        ###############################
        for i, token in enumerate(tok_sentence):
            
            #ann = which_ann(str(token), tok_sentence, i, tok_collection)
            
#             print((token) + '     ' + ann_info[ann_num_t_sen[i]])
            text_file.write((token) + '     ' + ann_info[ann_num_t_sen[i]])
            text_file.write("\n")
            
        # one sentence over...
#         print(("\n"))
        text_file.write("\n")

#         point_cnt += 1  
#         if point_cnt == 1:
#             break

#print(test_cnt)

In [7]:
# def which_ann(token, tok_sentence, i, tok_collection):
    
#     ann_info = {0: '/component/', 1: '/refinement_of_component/', 2: '/action/', 3: '/refinement_of_action/',
#                 4: '/condition/', 5: '/priority/', 6: '/motivation/', 7: '/role/', 8: '/object/', 9: '/refinement_of_object/',
#                 10: '/sub_action/', 11: '/sub_argument_of_action/', 12: '/sub_priority/', 13: '/sub_role/', 14: '/sub_object/',
#                 15: '/sub_refinement_of_object/', 16: '/none/'}
    
#     check_assign = False
#     result_ann_num = 16
#     check_3gram_right = False
#     check_3gram_left = False 
#     check_2gram_right = False
#     check_2gram_left = False     
    
#     evenif_3gram = False
#     evenif_2gram = False
#     #print(tok_collection)
    
#     for m, tok_array in enumerate(tok_collection):
#         for n, tok in enumerate(tok_array):
            
#             # Json 포맷에 annotation은 배열과 string으로 구분된다.
#             # 따라서, array와 string을 구분지어서 처리해준다.
#             # 3-gram까지 보기 때문에 정확도가 매우 높을 것이다.
            
#             ##### In case of 'array': 
#             #####
#             ########################
#             if type(tok) == type([]):
#                 #log('[[[array]]] --------------------------- len(tok_array)=', len(tok_array), 'n=', n)
            
#                 for l, _ in enumerate(tok): # array이므로, element들을 모두 순회해야 한다.
#                     #print('[[[array_element]]] --------------------------- len(tok)=', len(tok), 'l=', l)
#                     ### 1단계: 3-gram check
#                     #######################
#                     if len(tok) >= 3: # 최소 조건
#                         log('array = start len(tok_array) >= 3')
#                         ##### 1-1단계: RIGHT DIRECTION
#                         ### In terms of annotated token
#                         # last-1
#                         if l == len(tok)-2:
#                             check_3gram_right = False
#                         # last
#                         elif l == len(tok)-1:
#                             check_3gram_right = False
#                         ### In terms of sentence token
#                         # last-1
#                         elif i == len(tok_sentence)-2:
#                             check_3gram_right = False
#                         # last
#                         elif i == len(tok_sentence)-1:
#                             check_3gram_right = False
#                         else:
#                             check_3gram_right = True
                            
#                         ##### 1-2단계: LEFT DIRECTION
#                         ### In terms of annotated token
#                         # first-1
#                         if l == 1:
#                             check_3gram_left = False
#                         # first
#                         elif l == 0:
#                             check_3gram_left = False
#                         ### In terms of sentence token
#                         # first-1
#                         elif i == 1:
#                             check_3gram_left = False
#                         # first
#                         elif i == 0:
#                             check_3gram_left = False
#                         else:
#                             check_3gram_left = True
                        
#                         ##### 마무리...
#                         if check_3gram_right == True and check_3gram_left == True:
#                             # then compare
#                             if tok_sentence[i] == tok[l]:
#                                 if tok_sentence[i+1] == tok[l+1] and tok_sentence[i+2] == tok[l+2]:
#                                     if tok_sentence[i-1] == tok[l-1] and tok_sentence[i-2] == tok[l-2]:
#                                         return ann_info[m]
#                                     else:
#                                         return ann_info[16] # annotation 길이가 3이상인데, 여기서 안되면, 나가리다... 
#                         elif check_3gram_right == True and check_3gram_left == False:
#                             if tok_sentence[i] == tok[l]:
#                                 if tok_sentence[i+1] == tok[l+1] and tok_sentence[i+2] == tok[l+2]:
#                                     return ann_info[m]
#                                 else:
#                                     return ann_info[16]
#                         elif check_3gram_right == False and check_3gram_left == True:
#                             if tok_sentence[i] == tok[l]:
#                                 if tok_sentence[i-1] == tok[l-1] and tok_sentence[i-2] == tok[l-2]:
#                                     return ann_info[m]
#                                 else:
#                                     return ann_info[16]

#                         log('array = over len(tok_array) >= 3')
                        

#                     ### 1단계: 2-gram check
#                     #######################
#                     if len(tok) >= 2: # 최소 조건
#                         log('array = start len(tok_array) >= 2')    
#                         ##### 2-1단계: RIGHT DIRECTION
#                         ### In terms of annotated token
#                         # last
#                         if l == len(tok)-1:
#                             check_2gram_right = False                        
#                         ### In terms of sentence token
#                         # last
#                         elif i == len(tok_sentence)-1:
#                             check_2gram_right = False
#                         else:
#                             check_2gram_right = True                        
                        
#                         ##### 2-2단계: LEFT DIRECTION
#                         ### In terms of annotated token
#                         # first
#                         if l == 0:
#                             check_2gram_left = False                        
#                         ### In terms of sentence token
#                         # first                      
#                         elif i == 0:
#                             check_2gram_left = False
#                         else:
#                             check_2gram_left = True                        
                        
#                         ##### 마무리...
#                         if check_2gram_right == True and check_2gram_left == True:
#                             # then compare
#                             if tok_sentence[i] == tok[l]:
#                                 if tok_sentence[i+1] == tok[l+1]:
#                                     if tok_sentence[i-1] == tok[l-1]:
#                                         return ann_info[m]
#                                     else:
#                                         return ann_info[16]
#                         elif check_2gram_right == True and check_2gram_left == False:
#                             if tok_sentence[i] == tok[l]:
#                                 if tok_sentence[i+1] == tok[l+1]:
#                                     return ann_info[m]
#                                 else:
#                                     return ann_info[16]
#                         elif check_2gram_right == False and check_2gram_left == True:
#                             if tok_sentence[i] == tok[l]:
#                                 if tok_sentence[i-1] == tok[l-1]:
#                                     return ann_info[m]
#                                 else:
#                                     return ann_info[16]
                                                
#                         log('array = over len(tok_array) >= 2')
                                
#                     ### 3단계: 1-gram check
#                     ######################
#                     if len(tok) >= 1:
#                         log('array = start len(tok_array) >= 1')
#                         if tok_sentence[i] == tok[l]:
#                             return ann_info[m]
                        
#                         log('array = over len(tok_array) >= 1')
                           
                        
                   
#             # 위의 array 방법과 똑같이...            
#             ##### In case of 'string'
#             #####
#             ###########################################################################################################
#             else:
#                 #print('tok-is-string-', tok_array, len((tok_array)), type(tok), tok)
#                 check_3gram_right = False
#                 check_3gram_left = False 
#                 check_2gram_right = False
#                 check_2gram_left = False
#                 evenif_3gram = False
#                 evenif_2gram = False
#                 l = n
#                 #print('[[[string]]] --------------------------- len(tok_array)=', len(tok_array), 'n=', n)
                
#                 ### 1단계: 3-gram check
#                 #######################
#                 if len(tok_array) >= 3: # 최소 조건
#                     log('string = start len(tok_array) >= 3')
#                     ##### 1-1단계: RIGHT DIRECTION
#                     ### In terms of annotated token
#                     # last-1
#                     if l == len(tok_array)-2:
#                         check_3gram_right = False
#                     # last
#                     elif l == len(tok_array)-1:
#                         check_3gram_right = False
#                     ### In terms of sentence token
#                     # last-1
#                     elif i == len(tok_sentence)-2:
#                         check_3gram_right = False
#                     # last
#                     elif i == len(tok_sentence)-1:
#                         check_3gram_right = False
#                     else:
#                         check_3gram_right = True
                            
#                     ##### 1-2단계: LEFT DIRECTION
#                     ### In terms of annotated token
#                     # first-1
#                     if l == 1:
#                         check_3gram_left = False
#                     # first
#                     elif l == 0:
#                         check_3gram_left = False
#                     ### In terms of sentence token
#                     # first-1
#                     elif i == 1:
#                         check_3gram_left = False
#                     # first
#                     elif i == 0:
#                         check_3gram_left = False
#                     else:
#                         check_3gram_left = True
                        
#                     ##### 마무리...
#                     if check_3gram_right == True:
#                         if check_3gram_left == True:
#                             # then compare
#                             if tok_sentence[i] == tok_array[l]:
#                                 if tok_sentence[i+1] == tok_array[l+1] and tok_sentence[i+2] == tok_array[l+2]:
#                                     if tok_sentence[i-1] == tok_array[l-1] and tok_sentence[i-2] == tok_array[l-2]:
#                                         return ann_info[m]
#                                     else:
#                                         return ann_info[16]
#                     elif check_3gram_right == True and check_3gram_left == False:
#                         if tok_sentence[i] == tok_array[l]:
#                             if tok_sentence[i+1] == tok_array[l+1] and tok_sentence[i+2] == tok_array[l+2]:
#                                 return ann_info[m]
#                             else:
#                                 return ann_info[16]
#                     elif check_3gram_right == False and check_3gram_left == True:
#                         if tok_sentence[i] == tok_array[l]:
#                             if tok_sentence[i-1] == tok_array[l-1] and tok_sentence[i-2] == tok_array[l-2]:
#                                 return ann_info[m]
#                             else:
#                                 return ann_info[16]
                       
#                     log('string = over len(tok_array) >= 3')
                
#                 ### 1단계: 2-gram check
#                 #######################
#                 if len(tok_array) >= 2: # 최소 조건
#                     log('string = start len(tok_array) >= 2')  
#                     ##### 2-1단계: RIGHT DIRECTION
#                     ### In terms of annotated token
#                     # last
#                     if l == len(tok_array)-1:
#                         check_2gram_right = False                        
                    
#                     ### In terms of sentence token
#                     # last
#                     elif i == len(tok_sentence)-1:
#                         check_2gram_right = False
#                     else:
#                         check_2gram_right = True                        
                        
#                     ##### 2-2단계: LEFT DIRECTION
#                     ### In terms of annotated token
#                     # first
#                     if l == 0:
#                         check_2gram_left = False                        
#                     ### In terms of sentence token
#                     # first                      
#                     elif i == 0:
#                         check_2gram_left = False
#                     else:
#                         check_2gram_left = True                        
                        
                    
#                     ##### 마무리...
#                     if check_2gram_right == True and check_2gram_left == True:
#                         # then compare
#                         if tok_sentence[i] == tok_array[l]:
#                             if tok_sentence[i+1] == tok_array[l+1]:
#                                 if tok_sentence[i-1] == tok_array[l-1]:
#                                     return ann_info[m]
#                                 else:
#                                     return ann_info[16]
#                     elif check_2gram_right == True and check_2gram_left == False:
#                         if tok_sentence[i] == tok_array[l]:
#                             if tok_sentence[i+1] == tok_array[l+1]:
#                                 return ann_info[m]
#                             else:
#                                 return ann_info[16]
#                     elif check_2gram_right == False and check_2gram_left == True:
#                         if tok_sentence[i] == tok_array[l]:
#                             if tok_sentence[i-1] == tok_array[l-1]:
#                                 return ann_info[m]
#                             else:
#                                 return ann_info[16]
                                
#                     log('string = over len(tok_array) >= 2')                
#                 ### 3단계: 1-gram check
#                 ######################
#                 if len(tok_array) >= 1:
#                     log('string = start len(tok_array) >= 1')
#                     if tok_sentence[i] == tok_array[l]:
#                         return ann_info[m]
                    
#                 log('string = over len(tok_array) >= 1') 
    
#     # 위의 다돌았는데도 없으면...
#     # ann 안되는 것으로 간주하고...
#     return ann_info[16]

In [8]:
# import logging, sys
# logging.basicConfig(stream=sys.stderr, level=logging.DEBUG)
# logging.debug('A debug message!')
# logging.info('We processed %d records')

In [9]:
# logger = logging.getLogger()
# logger.setLevel(logging.DEBUG)

# logger.critical('This is a critical message.')
# logger.error('This is an error message.')
# logger.warning('This is a warning message.')
# logger.info('This is an informative message.')
# logger.debug('This is a low-level debug message.')

In [10]:
# def which_ann(token, tok_sentence, i, tok_collection):
    
#     ann_info = {0: '/component/', 1: '/refinement_of_component/', 2: '/action/', 3: '/refinement_of_action/',
#                 4: '/condition/', 5: '/priority/', 6: '/motivation/', 7: '/role/', 8: '/object/', 9: '/refinement_of_object/',
#                 10: '/sub_action/', 11: '/sub_argument_of_action/', 12: '/sub_priority/', 13: '/sub_role/', 14: '/sub_object/',
#                 15: '/sub_refinement_of_object/', 16: '/none/'}
    
#     num_assign = 0
#     result_ann_num = 16
    
#     for m, tok_array in enumerate(tok_collection):
#         for n, tok in enumerate(tok_array):
            
#             if type(tok) == type([]): 
#                 for l, ttt in enumerate(tok):
#                     if token == ttt:
#                         num_assign += 1
#                         result_ann_num = m
#             else:
#                 if token == tok:
#                     num_assign += 1
#                     result_ann_num = m
    
#     if num_assign == 1: # 1개만 assign되면 그대로 해당 annotation을 할당
#         return ann_info[result_ann_num]
#     elif num_assign == 0: # 1개도 assign되지 않으면 annotation안되는 것으로 판단해 none으로 처리
#         return '/none/'
#     else: # 2개이상 assign되면 2-gram을 사용해 재검사를 실시
#         num_assign = 0
#         for m, tok_array in enumerate(tok_collection):
#             for n, tok in enumerate(tok_array):
                
#                 # In case of 'array'
#                 if type(tok) == type([]): 
#                     for l, ttt in enumerate(tok): # because of array
#                         if i != len(tok_sentence)-1 and l != len(tok)-1: # not last token               
#                             if token == ttt and tok_sentence[i+1] == tok[l+1]:
#                                 #print(ttt)
#                                 num_assign += 1
#                                 result_ann_num = m
#                         elif i != 0 and l != 0: # not first token
#                             if token == ttt and tok_sentence[i-1] == tok[l-1]:
#                                 num_assign += 1
#                                 result_ann_num = m

#                 # In case of 'string'
#                 elif type(tok) == type('string'):
#                     if i != len(tok_sentence)-1 and n != len(tok_array)-1: # not last token
#                         if token == tok and tok_sentence[i+1] == tok_array[n+1]:
#                             num_assign += 1
#                             result_ann_num = m
#                     elif i != 0 and n != 0: # not first token
#                         if token == tok and tok_sentence[i-1] == tok_array[n-1]:
#                             num_assign += 1
#                             result_ann_num = m                    
                    
#         return ann_info[result_ann_num]
    
#         if num_assign >= 2:
#             return '/test/' # check how many /test/ are...

In [11]:
# def untokenize(words):
#     """
#     Untokenizing a text undoes the tokenizing operation, restoring
#     punctuation and spaces to the places that people expect them to be.
#     Ideally, `untokenize(tokenize(text))` should be identical to `text`,
#     except for line breaks.
#     """
#     text = ' '.join(words)
#     step1 = text.replace("`` ", '"').replace(" ''", '"').replace('. . .',  '...')
#     step2 = step1.replace(" ( ", " (").replace(" ) ", ") ")
#     step3 = re.sub(r' ([.,:;?!%]+)([ \'"`])', r"\1\2", step2)
#     step4 = re.sub(r' ([.,:;?!%]+)$', r"\1", step3)
#     step5 = step4.replace(" '", "'").replace(" n't", "n't").replace(
#          "can not", "cannot").replace(" //www", "//www")
#     step6 = step5.replace(" ` ", " '")
#     return step6.strip()

# tokenized = word_tokenize(EXAMPLE_TEXT)